__чистим данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
from datetime import datetime as dtm
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

---

In [2]:
!ls -1 data/raw

avito_2022-08-26_17-29_raw.xlsx
avito_2022-08-29_13-16_raw.xlsx
avito_2022-08-31_17-42_raw.xlsx
avito_2022-09-01_14-07_raw.xlsx
avito_2022-09-02_13-04_raw.xlsx
avito_2022-09-05_14-02_raw.xlsx


In [3]:
# каталог с файлами объявлений
archive_raw_path = 'data/raw/'

# файлым объявлений в хронологическом порядке
archive_raw_files = [
    #'avito_2022-08-26_17-29_raw.xlsx',
    #'avito_2022-08-29_13-16_raw.xlsx',
    #'avito_2022-08-31_17-42_raw.xlsx',
    'avito_2022-09-02_13-04_raw.xlsx',
    'avito_2022-09-05_14-02_raw.xlsx',
]

In [37]:
!ls -1 data/location

avito_2022-09-01_14-07_location.pkl
avito_2022-09-02_13-04_location.pkl
avito_2022-09-05_14-02_location.pkl
map_addr_unique.csv


In [8]:
# старая таблица адресов
loc_path = 'data/location/'
# loc_file = 'avito_2022-09-02_13-04_location.pkl'
loc_file = 'avito_2022-09-05_14-02_location.pkl'

In [51]:
result_path = 'data/'

## собираем и чистим данные

In [4]:
# pd.read_excel(archive_raw_path+archive_raw_files[1])

In [5]:
data = [ pd.read_excel(archive_raw_path+f) for f in archive_raw_files ]
ts = [ dtm.strptime(f,'avito_%Y-%m-%d_%H-%M_raw.xlsx') for f in archive_raw_files ]
for i in range(len(data)): data[i]['ts'] = ts[i]
data = pd.concat(data).drop_duplicates().reset_index(drop=True)
print('data:',len(data))
data.sample(3)

data: 7896


,avito_id,title,price,obj_name,adr,description,avito_page,ts
1563,2528321786,"Объявление «Квартира-студия, 39 м², 1/6 эт.» 5...",6200000,NaN,"ул. Челюскинцев, 57/3","Продам 2 х комнатную студию. Две комнаты, одна...",32,2022-09-02 13:04:00
3683,2130401998,"Объявление «3-к. квартира, 54 м², 4/5 эт.» 21 ...",7200000,NaN,"ул. Циолковского, 19",Продам трёхкомнатную квартиру в г. Севастополе...,74,2022-09-02 13:04:00
1709,2491088519,"Объявление «2-к. квартира, 43,4 м², 5/5 эт.» 1...",6900000,NaN,"ул. Ефремова, 8",В продаже квартира у моря! \n\nУдачное местора...,35,2022-09-02 13:04:00


In [6]:
from lib.avito import AvitoDataCleanerRealty

data_ = AvitoDataCleanerRealty.transform(data)
print('data:',len(data_))
data_.sample(2)

data: 7896


,avito_id,title,price,obj_name,adr,description,avito_page,ts,nrooms,floor,...,area,is_studio,is_apartment,is_part,is_auction,is_openspace,is_roof,is_SNT,priceM,is_last_floor
2456,2372785663,"1-к. квартира, 26 м², 2/3 эт.",3913298,ЖК «Голубая Бухта»,"аллея Защитников 35-й Батареи, д. 11а",ЖК «Голубая Бухта» — это неповторимое сочетани...,50,2022-09-02 13:04:00,1,2,...,26.0,False,False,False,False,False,False,False,3.913298,False
1386,2551560947,"4-к. квартира, 86 м², 2/9 эт.",10500000,NaN,"улица Косарева, 27",Продам 4-х комнатную квартиру ул Косарева 27. ...,28,2022-09-02 13:04:00,4,2,...,86.0,False,False,False,False,False,False,False,10.500000,False


## ищем геопозицию

### обновляем таблицу адресов

In [9]:
# дополняем таблицу адресов
cols = ['adr','latitude','longitude','truncated',]
loc = (
    data_[['adr']]
    .drop_duplicates()    
    .merge( pd.read_pickle(loc_path+loc_file), on=['adr'], how='outer' )[cols]
)
print(len(loc))
loc.sample(3)

1712


,adr,latitude,longitude,truncated
159,"Красивая улица , 38",44.585264,33.473158,False
858,"улица Загордянского, 24",44.633265,33.519180,False
1033,"Партизанская улица , 15",44.609627,33.513854,False


In [10]:
# адреса без геопозиции
loc_ = loc[ loc['latitude'].isnull() ][['adr']].reset_index(drop=True)
print(len(loc_))
loc_.sample(3)

166


,index,adr
159,1705,"Севастополь, проспект Генерала Острякова, 21"
5,377,"улица Александра Шостака, 1"
59,1605,"улица Героев Бреста, 44Г"


In [44]:
assert len(loc_)>0

In [11]:
import re

symb_lat = 'yexapocEXAPOCTHKBM'
symb_rus = 'уехаросЕХАРОСТНКВМ'
l2r = str.maketrans(symb_lat,symb_rus) 

# чистим строку адреса
loc_['adr_'] = (
        loc_['adr']
        .apply(lambda s: s.translate(l2r) )
        .apply(lambda s: re.sub(r'\b[Сс]евастополь\b','',s))
        .apply(lambda s: re.sub(r'\bмикрорайон\b',' ',s))
        .apply(lambda s: re.sub(r'\bмуниципальный округ\b',' район ',s))
        .apply(lambda s: re.sub(r'\bкотт?еджный пос[ёе]лок\b',' ',s))
        .apply(lambda s: re.sub(r'\bпос[ёе]лок\b',' ',s))
        .apply(lambda s: re.sub(r'^','Севастополь, ',s))
        # .apply(lambda s: re.sub(r'[Сс]евастополь.*село\b','Крым, село ',s))
        .apply(lambda s: re.sub(r',\s*,',', ',s))
        .apply(lambda s: re.sub(r' +',' ',s))
    )

In [12]:
%%time

# ищем геопозицию через OpenStreetMap

from geopy.geocoders import Nominatim

user_agent='Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
geolocator = Nominatim(user_agent=user_agent)

def get_coo(adr, geolocator=geolocator):
    adr_ = adr.split(',')
    loc = geolocator.geocode(adr_) # пробуем определить координаты
    truncated = False
    while True:
        if not (loc is None): # нашел...
            return {'latitude':loc.latitude,'longitude':loc.longitude,'truncated':truncated }#,adr_
        
        # ...не нашел...
        if( len(adr_) < 3 ): break # адрес слишком короткий для сокращения
        adr_ = adr_[:-1] # выкидываем часть адреса (номер дома)
        loc = geolocator.geocode( adr_) # ... и пробуем ещё раз
        truncated = True
    
    return {'latitude':None,'longitude':None,'truncated':None, }


# ищем геометку по адресу
loc_ = pd.concat([loc_, loc_['adr_'].progress_apply(get_coo).apply(pd.Series)],axis=1)
loc_.sample(4)

  0%|          | 0/166 [00:00<?, ?it/s]

CPU times: user 2.14 s, sys: 190 ms, total: 2.33 s
Wall time: 2min 13s


,index,adr,adr_,latitude,longitude,truncated
132,1678,"село Вилино, улица Чапаева, 19","Севастополь, село Вилино, улица Чапаева, 19",NaN,NaN,None
138,1684,"улица Гоголя, 22","Севастополь, улица Гоголя, 22",44.593940,33.518450,False
142,1688,"Севастополь, улица Правды, 33Б, подъезд 2","Севастополь, улица Правды, 33Б, подъезд 2",44.575405,33.440226,True
26,1199,"коттеджный посёлок Вилларис дель Мар, 103","Севастополь, Вилларис дель Мар, 103",NaN,NaN,None


In [21]:
print( 'было без геометки:',len(loc_) )
print( 'нашли геометок:', len( loc_[~loc_['latitude'].isnull()]) )
print( 'осталось без геометок:', len( loc_[loc_['latitude'].isnull()]) )

было без геометки: 166
нашли геометок: 126
осталось без геометок: 40


In [29]:
cols = [ 'adr','latitude','longitude','truncated' ]
pd.concat([loc.dropna()[cols],loc_.dropna()[cols],]).drop_duplicates().reset_index(drop=True)

,adr,latitude,longitude,truncated
0,"улица Генерала Крейзера, д. 8, корп. 5",44.604490,33.512078,True
1,"проспект Победы, 29В",44.584041,33.563608,False
2,"улица Военных Строителей, стр. 6.3",44.567946,33.405590,True
4,"улица Генерала Мельника, 11А/1",44.576784,33.565291,True
5,"шоссе Лабораторное, д. 33, секц. 6",44.574609,33.580993,True
...,...,...,...,...
161,"улица Гоголя, 20Б",44.594636,33.518431,False
162,"улица Горпищенко, 3",44.602131,33.548830,False
163,"Балаклава, улица Крестовского, 75",44.517256,33.606330,False
164,"Севастополь, улица Горпищенко, 143 к4",44.579399,33.578578,False


In [32]:
ts = data['ts'].max().strftime('%Y-%m-%d_%H-%M')
ts

'2022-09-05_14-02'

In [36]:
loc.to_pickle(f'{loc_path}/avito_{ts}_location.pkl')

## дополняем данные геометкой

In [46]:
data_ = data_.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

In [49]:
print('всего записей:', len( data_) )
print('записей без геометки:', len( data_[ data_['latitude'].isnull() ] ) )

всего записей: 7896
записей без геометки: 548


In [52]:
data_.to_pickle(result_path+'/data.pkl')

---

In [34]:
# loc_path+loc_file

In [ ]:
# читаем самый свежий файл объявлений

# f = archive_raw_files[-1]
# ts = dtm.strptime(f,'avito_%Y-%m-%d_%H-%M_raw.xlsx')

# data = pd.read_excel(archive_raw_path+f)
# data['ts'] = ts

# print(len(data))
# data.sample(3)

## докидываем старые объявления
#
# for f in archive_raw_files[-2::-1]:
#     print(f)
#     print('data:',len(data))
#     data_index = data[['avito_id','title']].drop_duplicates()
#     print('data index:',len(data_index))

#     ts = dtm.strptime(f,'avito_%Y-%m-%d_%H-%M_raw.xlsx')
#     data_ = (
#             pd.read_excel(archive_raw_path+f)    
#             .merge( data_index, on=['avito_id','title'],how='left',indicator=True )
#             .query('_merge=="left_only"')
#             .reset_index(drop=True)
#         )
#     data_['ts'] = ts
#     print('data new:', len(data_))
#     data = pd.concat([data_,data]).reset_index(drop=True)


# print('data:',len(data))
# print('avito_id:',len(data['avito_id'].drop_duplicates() ) )
# print('avito_id title:',len(data[['avito_id','title']].drop_duplicates() ) )